###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Educación Nacional

In [1]:
import pandas as pd

###Descarga del dataset desde el endpoint - archivo nudc-7mev.csv

Contiene información estadística (indicadoares educativos) de los niveles preescolar, básica y media relacionada con indicadores sectoriales por Municipio sin atípicos, desde el 2011 hasta 2023.

In [2]:
educational_indicators_mineducacion = pd.read_csv("https://www.datos.gov.co/resource/nudc-7mev.csv?$limit=5000000", dtype={'c_digo_municipio': str})
educational_indicators_mineducacion.head(3)

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media
0,2023,25843,Villa de San Diego de Ubate,25,Cundinamarca,3785.0,Cundinamarca (ETC),9747,97.97,97.80,...,5.28,0.53,3.15,8.88,4.82,7.52,2.82,6.12,11.70,3.14
1,2023,99624,Santa Rosalía,99,Vichada,3832.0,Vichada (ETC),1159,83.87,83.87,...,9.82,0.00,7.69,17.89,2.83,13.99,0.00,16.48,17.01,2.83
2,2023,99524,La Primavera,99,Vichada,3832.0,Vichada (ETC),2659,95.94,95.94,...,10.23,0.00,13.46,8.91,4.80,13.98,1.55,19.37,11.80,1.48


###Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
educational_indicators_mineducacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14585 entries, 0 to 14584
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   a_o                          14585 non-null  int64  
 1   c_digo_municipio             14585 non-null  object 
 2   municipio                    14585 non-null  object 
 3   c_digo_departamento          14585 non-null  int64  
 4   departamento                 14585 non-null  object 
 5   c_digo_etc                   14585 non-null  float64
 6   etc                          14585 non-null  object 
 7   poblaci_n_5_16               14579 non-null  object 
 8   tasa_matriculaci_n_5_16      14470 non-null  float64
 9   cobertura_neta               14474 non-null  float64
 10  cobertura_neta_transici_n    14533 non-null  float64
 11  cobertura_neta_primaria      14494 non-null  float64
 12  cobertura_neta_secundaria    14491 non-null  float64
 13  cobertura_neta_m

- Descripcion de cada columna

In [4]:
column_mapping = {
    'a_o': 'Vigencia del indicador',
    'c_digo_municipio': 'Código DANE del Municipio',
    'municipio': 'Nombre del Municipio',
    'c_digo_departamento': 'Código DANE del Departamento',
    'departamento': 'Nombre del Departamento',
    'c_digo_etc': 'Código DANE de la ETC',
    'etc': 'Nombre de la ETC',
    'poblaci_n_5_16': 'Población en edad teórica de estudiar (5 a 16 años) según proyecciones de población del DANE',
    'tasa_matriculaci_n_5_16': 'Proporción de la población entre 5 y 16 años que se encuentra asistiendo al sistema educativo. Cuando las proyecciones de población del DANE no capturan adecuadamente los flujos migratorios internos, puede alcanzar valores mayores al 100%.',
    'cobertura_neta': 'Es la relación entre el número de estudiantes matriculados en transición, primaria, secundaria y media que tienen la edad teórica (5 a 16 años) y el total de la población correspondiente a esa misma edad. Cuando las proyecciones de población del DANE no capturan adecuadamente los flujos migratorios internos, puede alcanzar valores mayores al 100%',
    'cobertura_neta_transici_n': 'Es la relación entre el número de estudiantes matriculados en transición que tienen la edad teórica para cursar este nivel (5 años) y el total de la población correspondiente a esa misma edad. Cuando las proyecciones de población del DANE no capturan adecuadamente los flujos migratorios internos, puede alcanzar valores mayores al 100%.',
    'cobertura_neta_primaria': 'Es la relación entre el número de estudiantes matriculados en primaria que tienen la edad teórica para cursar este nivel (6 a 10 años) y el total de la población correspondiente a esa misma edad. Cuando las proyecciones de población del DANE no capturan adecuadamente los flujos migratorios internos, puede alcanzar valores mayores al 100%.',
    'cobertura_neta_secundaria': 'Es la relación entre el número de estudiantes matriculados en secundaria que tienen la edad teórica para cursar este nivel (11 a 14 años) y el total de la población correspondiente a esa misma edad. Cuando las proyecciones de población del DANE no capturan adecuadamente los flujos migratorios internos, puede alcanzar valores mayores al 100%.',
    'cobertura_neta_media': 'Es la relación entre el número de estudiantes matriculados en media que tienen la edad teórica para cursar este nivel (15 a 16 años) y el total de la población correspondiente a esa misma edad. Cuando las proyecciones de población del DANE no capturan adecuadamente los flujos migratorios internos, puede alcanzar valores mayores al 100%.',
    'cobertura_bruta': 'Es la relación entre el número de estudiantes matriculados en transición, primaria, secundaria y media respecto a la población en edad teórica para cursar estos niveles (5 a 16 años). En algunos casos la demanda social es mayor a la población en edad teórica para cursar educación preescolar, básica y media (5 a 16 años), explicada por estudiantes en extraedad, por lo que el indicador toma valores superiores al 100%.',
    'cobertura_bruta_transici_n': 'Es la relación entre el número de estudiantes matriculados en transición respecto a la población en edad teórica para cursar este nivel (5 años). En algunos casos la demanda social es mayor a la población en edad teórica para cursar este nivel (5 años), explicada por estudiantes en extraedad, por lo que el indicador toma valores superiores al 100%.',
    'cobertura_bruta_primaria': 'Es la relación entre el número de estudiantes matriculados en primaria respecto a la población en edad teórica para cursar este nivel (6 a 10 años). En algunos casos la demanda social es mayor a la población en edad teórica para cursar este nivel (6 a 10 años), explicada por estudiantes en extraedad, por lo que el indicador toma valores superiores al 100%.',
    'cobertura_bruta_secundaria': 'Es la relación entre el número de estudiantes matriculados en secundaria respecto a la población en edad teórica para cursar este nivel (11 a 14 años). En algunos casos la demanda social es mayor a la población en edad teórica para cursar este nivel (11 a 14 años), explicada por estudiantes en extraedad, por lo que el indicador toma valores superiores al 100%.',
    'cobertura_bruta_media': 'Es la relación entre el número de estudiantes matriculados en media respecto a la población en edad teórica para cursar este nivel (15 a 16 años). En algunos casos la demanda social es mayor a la población en edad teórica para cursar este nivel (15 a 16 años), explicada por estudiantes en extraedad, por lo que el indicador toma valores superiores al 100%.',
    'tama_o_promedio_de_grupo': 'Corresponde al número promedio de estudiantes por grupo. Para su cálculo considera la matrícula y el número de grupos tanto del sector oficial como del no oficial',
    'sedes_conectadas_a_internet': 'Indica el porcentaje de sedes oficiales de Una Entidad Territorial Certificada que se encuentran conectadas a una red de Internet.',
    "Tasa de deserción intra - anual del sector oficial. Identifica la proporción de alumnos matriculados que por factores culturales, coyunturales o de prestación del servicio educativo, abandonan sus estudios durante el año lectivo.": "deserci_n",
    "Tasa de deserción intra - anual del sector oficial en transición. Identifica la proporción de alumnos matriculados que por factores culturales, coyunturales o de prestación del servicio educativo, abandonan sus estudios durante el año lectivo.": "deserci_n_transici_n",
    "Tasa de deserción intra - anual del sector oficial en primaria. Identifica la proporción de alumnos matriculados que por factores culturales, coyunturales o de prestación del servicio educativo, abandonan sus estudios durante el año lectivo.": "deserci_n_primaria",
    "Tasa de deserción intra - anual del sector oficial en secundaria. Identifica la proporción de alumnos matriculados que por factores culturales, coyunturales o de prestación del servicio educativo, abandonan sus estudios durante el año lectivo.": "deserci_n_secundaria",
    "Tasa de deserción intra - anual del sector oficial en media. Identifica la proporción de alumnos matriculados que por factores culturales, coyunturales o de prestación del servicio educativo, abandonan sus estudios durante el año lectivo.": "deserci_n_media",
    "Tasa de aprobación de estudiantes del sector oficial. Identifica el porcentaje de alumnos en educación preescolar, básica y media que aprueba de acuerdo con los planes y programas de estudio vigentes.": "aprobaci_n",
    "Tasa de aprobación de estudiantes del sector oficial en transición. Identifica el porcentaje de alumnos en este nivel educativo que aprueba de acuerdo con los planes y programas de estudio vigentes.": "aprobaci_n_transici_n",
    "Tasa de aprobación de estudiantes del sector oficial en primaria. Identifica el porcentaje de alumnos en este nivel educativo que aprueba de acuerdo con los planes y programas de estudio vigentes.": "aprobaci_n_primaria",
    "Tasa de aprobación de estudiantes del sector oficial en secundaria. Identifica el porcentaje de alumnos en este nivel educativo que aprueba de acuerdo con los planes y programas de estudio vigentes.": "aprobaci_n_secundaria",
    "Tasa de aprobación de estudiantes del sector oficial en media. Identifica el porcentaje de alumnos en este nivel educativo que aprueba de acuerdo con los planes y programas de estudio vigentes.": "aprobaci_n_media",
    "Tasa de reprobación de estudiantes del sector oficial. Identifica el porcentaje de alumnos en educación preescolar, básica y media que reprueba de acuerdo con los planes y programas de estudio vigentes.": "reprobaci_n",
    "Tasa de reprobación de estudiantes del sector oficial en transición. Identifica el porcentaje de alumnos en este nivel educativo que reprueba de acuerdo con los planes y programas de estudio vigentes.": "reprobaci_n_transici_n",
    "Tasa de reprobación de estudiantes del sector oficial en primaria. Identifica el porcentaje de alumnos en este nivel educativo que reprueba de acuerdo con los planes y programas de estudio vigentes.": "reprobaci_n_primaria",
    "Tasa de reprobación de estudiantes del sector oficial en secundaria. Identifica el porcentaje de alumnos en este nivel educativo que reprueba de acuerdo con los planes y programas de estudio vigentes.": "reprobaci_n_secundaria",
    "Tasa de reprobación de estudiantes del sector oficial en media. Identifica el porcentaje de alumnos en este nivel educativo que reprueba de acuerdo con los planes y programas de estudio vigentes.": "reprobaci_n_media",
    "Tasa de repitencia del sector oficial. Corresponde al porcentaje de alumnos matriculados en un año escolar en transición, primaria, secundaria y media que se encuentran repitiendo el mismo grado cursado el año anterior.": "repitencia",
    "Tasa de repitencia del sector oficial. Corresponde al porcentaje de alumnos matriculados en transición que se encuentran repitiendo el mismo grado cursado el año anterior.": "repitencia_transici_n",
    "Tasa de repitencia del sector oficial. Corresponde al porcentaje de alumnos matriculados en primaria que se encuentran repitiendo el mismo grado cursado el año anterior.": "repitencia_primaria",
    "Tasa de repitencia del sector oficial. Corresponde al porcentaje de alumnos matriculados en secundaria que se encuentran repitiendo el mismo grado cursado el año anterior.": "repitencia_secundaria",
    "Tasa de repitencia del sector oficial. Corresponde al porcentaje de alumnos matriculados en media que se encuentran repitiendo el mismo grado cursado el año anterior.": "repitencia_media"



}

In [5]:
len(column_mapping)

41

In [6]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in educational_indicators_mineducacion.columns:
  print(f'{col}: {educational_indicators_mineducacion[col].nunique()}')

a_o: 13
c_digo_municipio: 1271
municipio: 1037
c_digo_departamento: 34
departamento: 36
c_digo_etc: 194
etc: 100
poblaci_n_5_16: 8527
tasa_matriculaci_n_5_16: 5826
cobertura_neta: 5276
cobertura_neta_transici_n: 5021
cobertura_neta_primaria: 5269
cobertura_neta_secundaria: 5581
cobertura_neta_media: 5049
cobertura_bruta: 6147
cobertura_bruta_transici_n: 6286
cobertura_bruta_primaria: 6554
cobertura_bruta_secundaria: 6732
cobertura_bruta_media: 6628
tama_o_promedio_de_grupo: 4232
sedes_conectadas_a_internet: 1402
deserci_n: 1086
deserci_n_transici_n: 1157
deserci_n_primaria: 1001
deserci_n_secundaria: 1426
deserci_n_media: 1199
aprobaci_n: 2128
aprobaci_n_transici_n: 660
aprobaci_n_primaria: 1976
aprobaci_n_secundaria: 2903
aprobaci_n_media: 2196
reprobaci_n: 1569
reprobaci_n_transici_n: 660
reprobaci_n_primaria: 1487
reprobaci_n_secundaria: 2211
reprobaci_n_media: 1600
repitencia: 1401
repitencia_transici_n: 876
repitencia_primaria: 1495
repitencia_secundaria: 1770
repitencia_media: 96

In [7]:
# Eliminar columnas que se considera innecesarias para el proyecto
delete_columns = ['c_digo_departamento','etc', 'c_digo_etc']
# Eliminando las columnas especificadas
educational_indicators_mineducacion = educational_indicators_mineducacion.drop(delete_columns, axis=1)

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [8]:
# Obtener las columnas que no son numéricas
non_numeric_columns = ['c_digo_municipio', 'municipio', 'departamento']
print(non_numeric_columns)

['c_digo_municipio', 'municipio', 'departamento']


- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [9]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars =  [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [10]:
# Aplicar la función a todas las columnas categóricas
educational_indicators_mineducacion[non_numeric_columns] = educational_indicators_mineducacion[non_numeric_columns].astype(str)
for col in non_numeric_columns:
    educational_indicators_mineducacion[col] = educational_indicators_mineducacion[col].apply(remove_accents_and_special_chars)

In [11]:
# Imprimir categorías únicas para columnas de tipo object
for column in non_numeric_columns:
    print(f"Categorías en la columna '{column}':")
    print(educational_indicators_mineducacion[column].unique())
    print()

Categorías en la columna 'c_digo_municipio':
['25843' '99624' '99524' ... '5030' '0' '94663']

Categorías en la columna 'municipio':
['VILLA DE SAN DIEGO DE UBATE' 'SANTA ROSALIA' 'LA PRIMAVERA' ...
 'MEDELLIN' 'NACIONAL' 'MAPIRIPANA']

Categorías en la columna 'departamento':
['CUNDINAMARCA' 'VICHADA' 'VAUPES' 'GUAVIARE' 'GUAINIA' 'AMAZONAS'
 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA' 'PUTUMAYO'
 'CASANARE' 'ARAUCA' 'VALLE DEL CAUCA' 'TOLIMA' 'SUCRE' 'SANTANDER'
 'RISARALDA' 'QUINDIO' 'NORTE DE SANTANDER' 'NARINO' 'META' 'MAGDALENA'
 'LA GUAJIRA' 'HUILA' 'CHOCO' 'CORDOBA' 'CESAR' 'CAUCA' 'CAQUETA' 'CALDAS'
 'BOYACA' 'BOLIVAR' 'BOGOTA D.C.' 'ATLANTICO' 'ANTIOQUIA' 'NACIONAL'
 'ARCHIPIELAGO DE SAN ANDRES. PROVIDENCIA Y SANTA CATALINA']



- Estandarizar algunas columnas categoricas

In [12]:
# Columna

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigomunicipioatencion"

In [13]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


- Longitud de los códigos municipios en el df

In [14]:
# Convertir a strings
educational_indicators_mineducacion["c_digo_municipio"] = educational_indicators_mineducacion["c_digo_municipio"].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_mpio = educational_indicators_mineducacion["c_digo_municipio"].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los mpios tiene longitudes de :[5 4 1] caracteres


In [15]:
# Contar registros por longitud
long_df = longitud_mpio.value_counts().reset_index()
long_df.rename(columns={'cod_mpio': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitud_mpio))
long_df.head()

,c_digo_municipio,count,percentage
0,5,12806,0.878025
1,4,1776,0.121769
2,1,3,0.000206


In [16]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitud_mpio.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = educational_indicators_mineducacion[longitud_mpio == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['c_digo_municipio']])
    print()

Muestra de registros con longitud 5:
  c_digo_municipio
0            25843
1            99624
2            99524
3            99001
4            97889

Muestra de registros con longitud 4:
     c_digo_municipio
1121             5021
1123             5004
1124             5002
2078             5001
2097             8849

Muestra de registros con longitud 1:
     c_digo_municipio
2243                0
4384                0
5608                0



Nota: De lo anterior se notan claras inconsistencias en la columna 'c_digo_municipio' del df, el 87.8 tiene 5 dígitos (que corresponde con el codigo real que es de 5 dígitos), el 12.2% tiene solamente 4 dígitos (al parecer el cero a la izquierda de los códigos se suprimió) y hay 3 registros que solamente tienen un digito

- Verificar si los c_digo_municipio de 4 dígitos corresponden a departamentos que se identifican con el 0 + 1 dígito, para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [17]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtered = educational_indicators_mineducacion[educational_indicators_mineducacion['c_digo_municipio'].str.len() == 4]

# Obtener el primer dígito de cada código y convertirlo a una serie
first_digit = filtered['c_digo_municipio'].str[0]
# Obtener las categorías únicas de la columna 'departamento'
first_digit.unique()

array(['5', '8'], dtype=object)

El hecho de que los codigos de 4 cifras empiecen por 5 u 8 confirman nuestra sospecha de que efectivamente a los codigos de los departamentos de ANTIOQUIA Y ATLANTICO que son los unicos departamentos que tienen codigo Dane que inicia con un cero + un dígito, 05 y 08 respectivamente, se les suprimio el cero.

  - Adicionar un cero a los codigo_dane de 4 dígitos

In [18]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
educational_indicators_mineducacion['c_digo_municipio'] = educational_indicators_mineducacion['c_digo_municipio'].apply(add_zero_if_length_4)

In [19]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(educational_indicators_mineducacion[educational_indicators_mineducacion['c_digo_municipio'].str.len() == 4])

0

In [20]:
# Filtrar las filas donde 'codigo_dane' tiene 1 dígito
educational_indicators_mineducacion[educational_indicators_mineducacion['c_digo_municipio'].str.len() == 1]

,a_o,c_digo_municipio,municipio,departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,...,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media
2243,2021,0,NACIONAL,NACIONAL,9548263,0.9207,0.9190,0.6238,0.8817,0.7999,...,0.0806,0.0113,0.0659,0.1139,0.0783,0.0494,0.0148,0.0421,0.0723,0.0300
4384,2020,0,NACIONAL,NACIONAL,9540230,0.9216,0.9197,0.6436,0.8882,0.7940,...,0.0675,0.0209,0.0508,0.0989,0.0664,0.0552,0.0088,0.0509,0.0787,0.0303
5608,2019,0,NACIONAL,NACIONAL,9488850,0.9255,0.9235,0.6370,0.9003,0.7856,...,0.0578,0.0056,0.0459,0.0868,0.0503,0.0222,0.0064,0.0208,0.0309,0.0114


Nota: son indicadores que no se refieren a un municipio en particular sino a toda la nacion. Los eliminamos porque no interesan para el proyecto

In [21]:
# eliminar anteriores filas
educational_indicators_mineducacion = educational_indicators_mineducacion[educational_indicators_mineducacion['c_digo_municipio'].str.len() != 1]

In [22]:
# Verificar cuantos codigo_dane de 1 dígitos quedaron
len(educational_indicators_mineducacion[educational_indicators_mineducacion['c_digo_municipio'].str.len() == 1])

0

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [23]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [24]:
# Comparar listas de códigos
compare_lists(educational_indicators_mineducacion['c_digo_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en educational_indicators_mineducacion", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en educational_indicators_mineducacion que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


- Analizar cod de mpio que no corresponde a un cod real

In [25]:
mpio_cod_ = educational_indicators_mineducacion[educational_indicators_mineducacion['c_digo_municipio'].isin([ '94663'])]
mpio_cod_['municipio'].value_counts()

,count
municipio,
MAPIRIPANA,9


Nota:  Mapiripana figura como corregimiento de Barrancominas

In [26]:
# eliminar  filas
educational_indicators_mineducacion = educational_indicators_mineducacion[educational_indicators_mineducacion['c_digo_municipio'] != '94663']

- Verificar si cambio se aplicó

In [27]:
# Comparar listas de códigos
compare_lists(educational_indicators_mineducacion['c_digo_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en educational_indicators_mineducacion", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en educational_indicators_mineducacion que no están en Códigos de municipios dept_mpios_codes:
set()


- Verificar vigencias que cubre las informac.

In [28]:
educational_indicators_mineducacion['a_o'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011])

### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

- Eliminar columnas que dejaron de ser necesarias

In [29]:
# Eliminar columnas 'departamento', 'municipio' que ya no son necesarias
columns_to_drop = ['departamento', 'municipio']
educational_indicators_mineducacion = educational_indicators_mineducacion.drop(columns=columns_to_drop)

In [30]:
# Adicionar columna para trazabilidad de la fuente
# Lista de tuplas
educational_indicators_mineducacion['source_id'] = 118

In [31]:
educational_indicators_mineducacion.columns

Index(['a_o', 'c_digo_municipio', 'poblaci_n_5_16', 'tasa_matriculaci_n_5_16',
       'cobertura_neta', 'cobertura_neta_transici_n',
       'cobertura_neta_primaria', 'cobertura_neta_secundaria',
       'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n',
       'cobertura_bruta_primaria', 'cobertura_bruta_secundaria',
       'cobertura_bruta_media', 'tama_o_promedio_de_grupo',
       'sedes_conectadas_a_internet', 'deserci_n', 'deserci_n_transici_n',
       'deserci_n_primaria', 'deserci_n_secundaria', 'deserci_n_media',
       'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria',
       'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n',
       'reprobaci_n_transici_n', 'reprobaci_n_primaria',
       'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia',
       'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria',
       'repitencia_media', 'source_id'],
      dtype='object')

In [32]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'a_o':'year',
    'c_digo_municipio': 'dane_code',
    'poblaci_n_5_16': 'poblacion_5_16',
    'tasa_matriculaci_n_5_16': 'tasa_matricula_5_16',
    'cobertura_neta': 'cobertura_neta',
    'cobertura_neta_transici_n': 'cobertura_neta_transicion',
    'cobertura_neta_primaria': 'cobertura_neta_primaria',
    'cobertura_neta_secundaria': 'cobertura_neta_secundaria',
    'cobertura_neta_media': 'cobertura_neta_media',
    'cobertura_bruta': 'cobertura_bruta',
    'cobertura_bruta_transici_n': 'cobertura_bruta_transicion',
    'cobertura_bruta_primaria': 'cobertura_bruta_primaria',
    'cobertura_bruta_secundaria': 'cobertura_bruta_secundaria',
    'cobertura_bruta_media': 'cobertura_bruta_media',
    'tama_o_promedio_de_grupo': 'tamano_promedio_de_grupo',
    'sedes_conectadas_a_internet': 'sedes_conectadas_a_internet',
    'deserci_n': 'desercion',
    'deserci_n_transici_n': 'desercion_transicion',
    'deserci_n_primaria': 'desercion_primaria',
    'deserci_n_secundaria': 'desercion_secundaria',
    'deserci_n_media': 'desercion_media',
    'aprobaci_n': 'aprobacion',
    'aprobaci_n_transici_n': 'aprobacion_transicion',
    'aprobaci_n_primaria': 'aprobacion_primaria',
    'aprobaci_n_secundaria': 'aprobacion_secundaria',
    'aprobaci_n_media': 'aprobacion_media',
    'reprobaci_n': 'reprobacion',
    'reprobaci_n_transici_n': 'reprobacion_transicion',
    'reprobaci_n_primaria': 'reprobacion_primaria',
    'reprobaci_n_secundaria': 'reprobacion_secundaria',
    'reprobaci_n_media': 'reprobacion_media',
    'repitencia': 'repitencia',
    'repitencia_transici_n': 'repitencia_transicion',
    'repitencia_primaria': 'repitencia_primaria',
    'repitencia_secundaria': 'repitencia_secundaria',
    'repitencia_media': 'repitencia_media',
    'source_id': 'source_id'
}

# Renombrar las columnas
educational_indicators_mineducacion.rename(columns=translation_map, inplace=True)

In [33]:
# convertir columna a poblacion_5_16 a numerica
# Primero, eliminamos cualquier carácter no numérico (como comas, espacios, etc.)
educational_indicators_mineducacion['poblacion_5_16'] = educational_indicators_mineducacion['poblacion_5_16'].str.replace(r'\D', '', regex=True)

# Luego, convertimos la columna a tipo numérico (int64), manejando los valores NaN si existen
educational_indicators_mineducacion['poblacion_5_16'] = pd.to_numeric(educational_indicators_mineducacion['poblacion_5_16'], errors='coerce').astype('Int64')

In [34]:
educational_indicators_mineducacion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14573 entries, 0 to 14584
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   year                         14573 non-null  int64  
 1   dane_code                    14573 non-null  object 
 2   poblacion_5_16               14567 non-null  Int64  
 3   tasa_matricula_5_16          14458 non-null  float64
 4   cobertura_neta               14463 non-null  float64
 5   cobertura_neta_transicion    14521 non-null  float64
 6   cobertura_neta_primaria      14484 non-null  float64
 7   cobertura_neta_secundaria    14479 non-null  float64
 8   cobertura_neta_media         14481 non-null  float64
 9   cobertura_bruta              14505 non-null  float64
 10  cobertura_bruta_transicion   14476 non-null  float64
 11  cobertura_bruta_primaria     14492 non-null  float64
 12  cobertura_bruta_secundaria   14485 non-null  float64
 13  cobertura_bruta_media

In [35]:
educational_indicators_mineducacion.head()

,year,dane_code,poblacion_5_16,tasa_matricula_5_16,cobertura_neta,cobertura_neta_transicion,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,...,reprobacion_transicion,reprobacion_primaria,reprobacion_secundaria,reprobacion_media,repitencia,repitencia_transicion,repitencia_primaria,repitencia_secundaria,repitencia_media,source_id
0,2023,25843,9747,97.97,97.80,70.29,89.04,89.23,62.44,110.18,...,0.53,3.15,8.88,4.82,7.52,2.82,6.12,11.70,3.14,118
1,2023,99624,1159,83.87,83.87,64.94,85.74,54.05,27.75,93.10,...,0.00,7.69,17.89,2.83,13.99,0.00,16.48,17.01,2.83,118
2,2023,99524,2659,95.94,95.94,73.37,102.36,60.71,29.68,105.45,...,0.00,13.46,8.91,4.80,13.98,1.55,19.37,11.80,1.48,118
3,2023,99001,4641,144.52,144.52,114.44,162.78,78.48,33.46,158.50,...,0.00,12.96,18.84,8.81,11.18,0.82,13.25,10.45,6.39,118
4,2023,97889,561,47.95,47.95,68.75,52.19,25.84,13.04,52.05,...,0.00,0.00,0.00,0.00,12.33,3.13,19.50,3.95,4.00,118


## Salvar en archivo csv en el drive

In [36]:
# Guardar en archivos CSV en el drive
educational_indicators_mineducacion.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/educational_indicators_mineducacion.csv', index=False)